<a href="https://colab.research.google.com/github/DevunuriSrinidhi/ExcelR/blob/main/Text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
dataset = pd.read_csv('hate_speech.csv')
dataset.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [7]:
dataset.shape

(5242, 3)

In [8]:
dataset.label.value_counts()

,count
label,
0,3000
1,2242


In [9]:
for index, tweet in enumerate(dataset["tweet"][10:15]):
  print(index+1,"-",tweet)

1 -  â #ireland consumer price index (mom) climbed from previous 0.2% to 0.5% in may   #blog #silver #gold #forex
2 - we are so selfish. #orlando #standwithorlando #pulseshooting #orlandoshooting #biggerproblems #selfish #heabreaking   #values #love #
3 - i get to see my daddy today!!   #80days #gettingfed
4 - ouch...junior is angryð#got7 #junior #yugyoem   #omg 
5 - i am thankful for having a paner. #thankful #positive     


In [10]:
import re
def clean_text(text):
  text = re.sub(r'[^a-zA-Z\']', '', text)
  text = re.sub(r'[^\x00-\x7F]+', '', text)
  text = text.lower()
  return text

In [11]:
dataset['class text'] = dataset.tweet.apply(lambda x: clean_text(x))

In [12]:
dataset.head(10)

,id,label,tweet,class text
0,1,0,@user when a father is dysfunctional and is s...,userwhenafatherisdysfunctionalandissoselfishhe...
1,2,0,@user @user thanks for #lyft credit i can't us...,useruserthanksforlyftcreditican'tusecausetheyd...
2,3,0,bihday your majesty,bihdayyourmajesty
3,4,0,#model i love u take with u all the time in ...,modeliloveutakewithuallthetimeinur
4,5,0,factsguide: society now #motivation,factsguidesocietynowmotivation
5,6,0,[2/2] huge fan fare and big talking before the...,hugefanfareandbigtalkingbeforetheyleavechaosan...
6,7,0,@user camping tomorrow @user @user @user @use...,usercampingtomorrowuseruseruseruseruseruseruse...
7,8,0,the next school year is the year for exams.ð...,thenextschoolyearistheyearforexamscan'tthinkab...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,wewonlovethelandallincavschampionsclevelandcle...
9,10,0,@user @user welcome here ! i'm it's so #gr...,useruserwelcomeherei'mit'ssogr


In [16]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
from nltk.corpus import stopwords
len(stopwords.words('english'))

179

In [18]:
stop = stopwords.words('english')

In [19]:
def gen_freq(text):
  word_list = []
  for word in text.split():
    word_list.extend(word)
  word_freq = pd.Series(word_list).value_counts()
  word_freq = word_freq.drop(stop, errors='ignore')
  return word_freq

In [20]:
def any_neg(words):
  for word in words:
    if word in ['no', 'nor', 'not'] or re.search(r"\wn't", word):
      return 1
  else:
    return 0

In [21]:
def any_rare(words, rare_100):
  for word in words:
    if word in rare_100:
      return 1
  else:
    return 0

In [22]:
def is_question(words):
  for word in words:
    if word in ['when', 'what', 'how', 'why', 'who', 'where']:
      return 1
  else:
    return 0

In [33]:

dataset['clean_text'] = dataset.tweet.apply(lambda x: clean_text(x))
word_freq = gen_freq(dataset.clean_text.str)
rare_100 = word_freq[-100:]
dataset['word_count']=dataset.clean_text.str.split().apply(lambda x:len(x))
dataset['any_neg'] = dataset.clean_text.str.split().apply(lambda x:any_neg(x))
dataset['any_rare'] = dataset.clean_text.str.split().apply(lambda x:any_rare(x,rare_100))
dataset['is_question'] = dataset.clean_text.str.split().apply(lambda x:is_question(x))
dataset['char_count'] = dataset.clean_text.apply(lambda x:len(x))